In [1]:
using Pkg; Pkg.activate(@__DIR__)
# Pkg.instantiate()

  Activating project at `~/Julia_ML_training/unit2`


---
title: "Unit 2 - Processing Data"
engine: julia
jupyter: julia-two-1.11
---

In this unit we focus on **data**. We start by considering basic Julia data structures including dictionaries, sets, named tuples, and others. We then then focus on basic text (string) processing in Julia. Then we move onto Dataframes - a general and useful way to keep tabular data. We then touch on JSON files, and serialization.

# Basic data structures

Beyond [arrays](https://docs.julialang.org/en/v1/base/arrays/) which are very important and include `Vector` and `Matrix`, here are some basic data structures in Julia:

## Dictionaries

*See [Dictionaries in the Julia docs](https://docs.julialang.org/en/v1/base/collections/#Dictionaries).*

Dictionaries (often called hash maps or associative arrays) store key-value pairs. Each key in a dictionary must be unique. They are incredibly useful for many purposes because their looking up values quickly based on a unique identifier. In particular, well designed hash maps are implemented with lookup (get value by key), insertion (insert value to key), and deletion (remove value by key) operations taking average $O(1)$ (constant) time^["Constant time" suffices in practice, there is minutiae and worst case is $O(n)$ which is bad -- for theoretical applications, they can be implemented in $O(\log n)$ worst case time with self-balancing trees, but all practical applications rely on constant time average and engineering tricks to avoid the linear time worst case.]. This makes them very popular both for their simplicity but also to speed up algorithms with smart tricks (like reverse indeces built in hash maps).

In [3]:
d = Dict()

Dict{Any, Any}()

In [ ]:
d[

In [4]:
pop = Dict()
pop["Australia"] = 27_864_000
pop["United States"] = 340_111_000
pop["Finland"] = 5_634_000

5634000

In [5]:
pop

Dict{Any, Any} with 3 entries:
  "United States" => 340111000
  "Finland"       => 5634000
  "Australia"     => 27864000

In [7]:
my_pair = "Australia" => 27_000_000

"Australia" => 27000000

In [8]:
typeof(my_pair)

Pair{String, Int64}

In [9]:
my_pair.first

"Australia"

In [10]:
my_pair.second

27000000

In [6]:
Dict(["Australia" => 27_000_000, "United States" => 340_000_000])

Dict{String, Int64} with 2 entries:
  "United States" => 340000000
  "Australia"     => 27000000

Infer its type:

In [ ]:
@show typeof(pop)

We can restrict the types:

In [ ]:
strict_pop = Dict{String,Int}()
strict_pop["Australia"] = 27_864_000
strict_pop["United States"] = 340_111_000
strict_pop["Finland"] = 5_634_000

strict_pop

In [ ]:
# this is okay
pop["North Pole"] = 0.5
# not okay
strict_pop["North Pole"] = 0.5

Checking and accessing dictionary values:

In [ ]:
# Accessing a value
population_australia = pop["Australia"]
println("Population of Australia: ", population_australia)

mars_pop_safe = get(pop, "Mars", nothing)

Use `haskey` to check if the key exists:

In [ ]:
if haskey(pop, "United States")
    println("United States population exists: ", pop["United States"])
end

if !haskey(pop, "Atlantis")
    println("Atlantis population does not exist.")
end

More useful operations:

* `keys()`: Returns an iterable collection of all keys in the dictionary.
* `values()`: Returns an iterable collection of all values in the dictionary.
* `pairs()`: Returns an iterable collection of `Pair` objects (key => value) for all entries.
* `length()`: Returns the number of key-value pairs in the dictionary.
* `empty!()`: Removes all key-value pairs from the dictionary.


In [ ]:
println()
println("Keys in pop: ", keys(pop))
println("Values in pop: ", values(pop))
println("Pairs in pop: ", pairs(pop))
println("Number of entries in pop: ", length(pop))

# Iterating through a dictionary
println()
println("Iterating through pop:")
for (country, population) in pop
    println("$country: $population")
end

# Create a dictionary using the Dict constructor with pairs
new_countries = Dict("Canada" => 38_000_000, "Mexico" => 126_000_000)
println()
println("New countries dictionary: ", new_countries)

# Note that `=>` constructs a pair:
typeof(:s => 2)

# Merging dictionaries (creates a new dictionary)
merged_pop = merge(pop, new_countries)
println("Merged population dictionary: ", merged_pop)

# In-place merge (modifies the first dictionary)
merge!(pop, new_countries)
println("Pop after in-place merge: ", pop)

# Clearing a dictionary
empty!(pop)
println("Pop after empty!: ", pop)

## Sets

See [Set-Like Collections in the Julia docs](https://docs.julialang.org/en/v1/base/collections/#Set-Like-Collections). Here are some examples.

In [ ]:
A = Set([2,7,2,3])
B = Set(1:6)
omega = Set(1:10)

AunionB = union(A, B)
AintersectionB = intersect(A, B)
BdifferenceA = setdiff(B,A)
Bcomplement = setdiff(omega,B)
AsymDifferenceB = union(setdiff(A,B),setdiff(B,A))
println("A = $A, B = $B")
println("A union B = $AunionB")
println("A intersection B = $AintersectionB")
println("B diff A = $BdifferenceA")
println("B complement = $Bcomplement")
println("A symDifference B = $AsymDifferenceB")
println("The element '6' is an element of A: $(in(6,A))")
println("Symmetric difference and intersection are subsets of the union: ",
        issubset(AsymDifferenceB,AunionB),", ", issubset(AintersectionB,AunionB))

Internally, sets are a thin wrapper around dictionaries with no values:

```julia
# base/set.jl
struct Set{T} <: AbstractSet{T}
    dict::Dict{T,Nothing}

    global _Set(dict::Dict{T,Nothing}) where {T} = new{T}(dict)
end
```

## Named tuples

In addition to tuples (see [docs](https://docs.julialang.org/en/v1/manual/types/#Tuple-Types)), Julia has [named tuples](https://docs.julialang.org/en/v1/manual/types/#Named-Tuple-Types). Here are some examples:


In [ ]:
my_stuff = (age=28, gender=:male, name="Aapeli")
yonis_stuff = (age=51, gender=:male, name="Yoni")

my_stuff.gender

Named tuples are also used as [keyword arguments](https://docs.julialang.org/en/v1/manual/functions/#Keyword-Arguments).

In [ ]:
function my_function_kwargs(; keyword_arg1=default_value1, keyword_arg2=default_value2)
    println("Keyword 1: $keyword_arg1")
    println("Keyword 2: $keyword_arg2")
end

todays_args = (keyword_arg1="hello!", keyword_arg2="nothing")
my_function_kwargs(; todays_args...)

An example with Plots:

In [ ]:
using Plots
using LaTeXStrings

# we can use named tuples to pass in keyword arguments
args = (label=false, xlim=(-1,1), xlabel=L"x")
# `...` is the "splat" operator, similar to `**args` in python
p1 = plot(x->sin(1/x); ylabel=L"\sin(\frac{1}{x})", args...)
p2 = plot(x->cos(1/x); ylabel=L"\cos(\frac{1}{x})", args...)
plot(p1, p2, size=(700,300))

## Structs (Composite Types)

You can obviously define your own types [see composite types in docs](https://docs.julialang.org/en/v1/manual/types/#Composite-Types). You can use `struct` which is by default immutable, or `mutable struct`. In terms of memory management, immutable types sit on the stack while mutable types sit on the heap and require allocations and garbage collection.

In [ ]:
struct Place
  name::String
  lon::Float64
  lat::Float64
end

In [ ]:
# Constructing Place instances
new_york = Place("New York", -74.0060, 40.7128)
brisbane = Place("Brisbane", 153.0251, -27.4698)
townsville = Place("Townsville", 146.8169, -19.2581)

println(new_york)
println(brisbane)
println(townsville)

# access fields
println("Latitude of new_york: ", new_york.lat)

We can also have constructors with logic

In [ ]:
"""
A fancier place that wraps longitude automatically
"""
struct FancyPlace
  name::String
  lon::Float64
  lat::Float64

  # Default constructor (provided automatically if no inner constructors are defined)
  function FancyPlace(name::String, lon::Float64, lat::Float64)
    # make sure longitude is in [-180,180)
    wrapped_lon = mod(lon + 180, 360) - 180
    # new is a special keyword used to create the actual struct instance
    # It takes the values for the fields in the order they are defined in
    # the struct, effectively calling the "primary" constructor
    new(name, wrapped_lon, lat)
  end

  # Custom constructor for an "unnamed" place
  FancyPlace(lon::Float64, lat::Float64) = FancyPlace("[unnamed]", lon, lat) # The `new` keyword calls the primary constructor
end

# Now we can use the new constructor
unnamed_location = FancyPlace(1000.0, 20.0)
println("\nUnnamed location: ", unnamed_location)
println("Name of unnamed_location: ", unnamed_location.name)

We can add additional "outer" constructors, but they cannot call `new` directly. For example, suppose you use a GIS package with your own coordinates

In [ ]:
struct WGS84Coordinates{T}
  x::T
  y::T
end

function FancyPlace(name::String, coords::WGS84Coordinates)
    return FancyPlace(name, Float64(coords.x), Float64(coords.y))
end

zero_coords = WGS84Coordinates{Float32}(142.2, 11.35)
mariana_trench = FancyPlace("Mariana Trench", zero_coords)

@show mariana_trench

The [Parameters.jl](https://github.com/mauro3/Parameters.jl) package extends the functionality by automatically creating keyword based constructors for struct beyond the default constructors.

In [ ]:
using Parameters

@with_kw struct MyStruct
    a::Int = 6
    b::Float64 = -1.1
    c::UInt8
end

MyStruct(c=4) # call to the constructor created with the @with_kw with a keyword argument

Another useful macro based modification of the language is with the [Accessors.jl](https://github.com/JuliaObjects/Accessors.jl) package. It allows to update values of structs (immutable) easily by creating a copy without having to copy all other values:

In [ ]:
using Accessors

a = MyStruct(a=10, c=4)
@show a

b = @set a.c = 0
@show b;

# but observe a is still untouched
@show a

## Datastructures (not in the standard library)

The [JuliaCollections](https://github.com/JuliaCollections) library provides other data structures. One useful package is [DataStructures.jl](https://github.com/JuliaCollections/DataStructures.jl). Let's use for example a heap for [heap sort](https://en.wikipedia.org/wiki/Heapsort) (note that this is only for illustrative purposes. The system's [sort](https://docs.julialang.org/en/v1/base/sort/) will be more efficient).


In [ ]:
using Random, DataStructures
Random.seed!(0)

function heap_sort!(a::AbstractArray)
    h = BinaryMinHeap{eltype(a)}()
    for e in a
        push!(h, e) #This is an O(log n) operation
    end

    #Write back onto the original array
    for i in 1:length(a)
        a[i] = pop!(h) #This is an O(log n) operation
    end
    return a
end

data = [65, 51, 32, 12, 23, 84, 68, 1]
heap_sort!(data)
@show data
@show heap_sort!(["Finland", "USA", "Australia", "Brazil"]);

Again, note that this is a bunch slower than the standard lib sort:

In [ ]:
using BenchmarkTools

numbers = rand(10_000);

In [ ]:
@benchmark sort!(numbers)

In [ ]:
@benchmark heap_sort!(numbers)

# Basic text processing

Here are [strings in the julia docs](https://docs.julialang.org/en/v1/manual/strings/). Let's see some examples:

In [ ]:
x = 2
"The value of x is $x"

In [ ]:
split("Hello world!")

In [ ]:
# multiline blocks will clear up whitespace to make life nice with indentation
my_life_story = """
    I was born
       in 1935.
    """

println(my_life_story)

In [ ]:
ismutable(String)

In [ ]:
println("A rough ASCII table")
println("Decimal\tHex\tCharacter")
for c in 0x20:0x7E
    println(c,"\t","0x" * string(c,base=16),"\t",Char(c))
end

## Regular Expressions

Julia has built-in regex!

In [ ]:
text = "Julia is fun!"
pattern = r"Julia"
occursin(pattern, text)   # true

In [ ]:
text = "Call me at 0468879289 when I'm home, or 0468879555 if I'm at work"
for m in eachmatch(r"04\d{8}", text)
  println("Found phone number $(m.match)")
end

## Reading and writing files

The `open` function is your primary tool, often used with `do` blocks to ensure files are automatically closed.

To **write** text to a file:

In [ ]:
open("work/my_output.txt", "w") do io
    write(io, "Hello from Julia!\n")
    write(io, "This is a second line.")
end

Here, `"w"` signifies "write mode." If the file doesn't exist, it's created; if it does, its contents are overwritten.

To **append** text to an existing file:

In [ ]:
open("work/my_output.txt", "a") do io
    write(io, "\nAppending a new line.")
end

The `"a"` mode means "append." New stuff is added to the end of the file.

To **read** the entire content of a file:

In [ ]:
file_content = read("work/my_output.txt", String)
println(file_content)

The `read` function with `String` as the type argument reads the whole file into a single string.

For reading a file line by line, which is more memory-efficient for large files:

In [ ]:
open("work/my_output.txt", "r") do io
    for line in eachline(io)
        println("Line: ", line)
    end
end

## Some extras

The [Printf](https://docs.julialang.org/en/v1/stdlib/Printf/) package is built-in and provides formatted output functions similar to the C standard library.

Strings are related to IO. See the [I/O and Network docs](https://docs.julialang.org/en/v1/base/io-network/). Something quite common is to use `flush(stdout)`.

Sometimes when writing test code we want strings to be approximately equal. For this it is useful to use the [StringDistances.jl](https://github.com/matthieugomez/StringDistances.jl) package.

Consider the [YAML.jl](https://github.com/JuliaData/YAML.jl) package for [YAML files](https://en.wikipedia.org/wiki/YAML).

# Dataframes

Dataframes are huge subject. The Julia [Dataframes.jl](https://github.com/JuliaData/DataFrames.jl) package provides functionality similar to [Python pandas](https://en.wikipedia.org/wiki/Pandas_(software)) or [R dataframes](https://www.geeksforgeeks.org/r-data-frames/).

Let's get started

In [ ]:
using DataFrames

## Constructing DataFrames

### From Column-Value Pairs

The most common way to create a DataFrame is by providing column names (as symbols) and their corresponding vectors of data.

In [ ]:
# Create a DataFrame with two columns 'a' and 'b'
df = DataFrame(a = [1, 2, 3], b = [2.0, 4.0, 6.0])

Notice that Julia infers the data types for each column. Here, `a` is `Int64` and `b` is `Float64`.

We can also create DataFrames using `Pairs`:

In [ ]:
DataFrame(:c => ["apple", "banana", "cherry"], :d => [true, false, true])

### From Dictionaries

You can also construct a DataFrame from a dictionary where keys are column names (symbols or strings) and values are vectors.

In [ ]:
DataFrame(Dict(
    :name => ["Aapeli", "Yoni", "Jesse"],
    :age => [25, 30, 35],
    :city => ["New York", "Brisbane", "Berlin"]
))

### From `NamedTuple`s

Creating a DataFrame from a vector of `NamedTuple`s is very flexible.

In [ ]:
DataFrame([
    (id = 1, value = 10.5, tag = "A"),
    (id = 2, value = 20.1, tag = "B"),
    (id = 3, value = 15.0, tag = "C")
])

If the `NamedTuple`s have different fields or different orders, we can use `Tables.dictcolumntable` to fill missing values with `missing`.

In [ ]:
DataFrame(Tables.dictcolumntable([
    (id = 1, name = "Julia"),
    (id = 2, score = 95.5),
    (id = 3, name = "DataFrame", type = "Table")
]))

Notice the `?` after the types, indicating that these columns now allow `missing` values.

## Column Names and Basic Information

In DataFrames.jl, columns are primarily accessed using `Symbol`s.

In [ ]:
df = DataFrame(a = [1, 2, 3], b = [2.0, 4.0, 6.0], c = ["x", "y", "z"])

df[:, :a]

You can get the column names:

In [ ]:
names(df)

And column types:

In [ ]:
eltype.(eachcol(df))

### Size and Dimensions

To get the dimensions of a DataFrame, similar to matrices:

In [ ]:
size(df) # (rows, columns)

You can also specify the dimension:

In [ ]:
@show size(df, 1) # Number of rows
@show size(df, 2) # Number of columns

### Column-based Storage and Iterators

DataFrames.jl stores data in a column-oriented fashion. This means each column is essentially a `Vector`.

You can retrieve a column using dot syntax or indexing:

In [ ]:
df.a # Access column 'a' using dot syntax
df[!, :b] # Access column 'b' using ! (returns a view, i.e., no copy)
df[:, :c] # Access column 'c' using :, which makes a copy

The difference between `.` and `!` versus `:` for column retrieval is crucial for performance and understanding data manipulation.

In [ ]:
df.a === df[!, :a] # They refer to the same underlying data

In [ ]:
df.a === df[:, :a] # The : operator creates a copy, so they are not the same object

When you need to iterate through rows, you can use `eachrow(df)`:

In [ ]:
for row in eachrow(df)
    println("Row: $(row.a), $(row.b), $(row.c)")
end

Each `row` here is a `DataFrameRow` object, which behaves like a `NamedTuple` for row-wise access.

## Indexing and Slicing

DataFrames can be indexed similar to matrices, but with the added flexibility of column names.

### Positional Indexing

In [ ]:
df[1, 1] # First row, first column
df[2, :b] # Second row, column 'b'
df[1, :] # First row (returns a DataFrameRow)
df[:, 1] # First column (returns a Vector, view)

### Column Selection

You can select multiple columns by passing a vector of column names (symbols or strings):

In [ ]:
df[:, [:a, :c]] # Select columns 'a' and 'c' (creates a new DataFrame)

Or exclude columns using `Not`:

In [ ]:
df[:, Not(:b)] # Select all columns except 'b'

You can combine `Not` with a vector of columns:

In [ ]:
df[:, Not([:a, :b])] # Select all columns except 'a' and 'b'

### Views vs. Copies

Recall the distinction between `!` and `:` for column access. This also applies to row and full DataFrame indexing.

- `df[!, :colname]` returns a *view* of the column (no copy).
- `df[:, :colname]` returns a *copy* of the column.
- `df[!, [col1, col2]]` returns a *view* of the selected columns (a `SubDataFrame`).
- `df[:, [col1, col2]]` returns a *copy* of the selected columns (a new `DataFrame`).
- `df[!, row_indices, col_indices]` returns a `SubDataFrame` (view).
- `df[row_indices, col_indices]` returns a new `DataFrame` (copy).

Using views (`!`) is more memory-efficient when you don't need a separate copy of the data and want changes to the view to reflect in the original DataFrame. However, views require translating between the parent df indeces and the view indeces, which might in theory cause performance issues in edge cases.

## Getting, Setting, and Mutating Data

You can retrieve, set, and modify individual cells, rows, or columns.

### Setting Individual Values

In [ ]:
df[1, :a] = 100 # Set value at row 1, column 'a'

### Setting Entire Columns

In [ ]:
df.b = [10.0, 20.0, 30.0] # Replace column 'b'

If the new column has a different type, it will be converted if possible, or an error will occur. If a column doesn't exist, it will be added.

In [ ]:
df.d = ["alpha", "beta", "gamma"] # Add a new column 'd'

### Broadcasting Assignment

Broadcasting (`.=`) is extremely powerful for performing element-wise operations and assignments efficiently.

In [ ]:
df.a .= 0 # Set all values in column 'a' to 0

You can also use it with a scalar or a vector of compatible size:

In [ ]:
df.b .= df.b * 2 # Double all values in column 'b'

Or apply a function:

In [ ]:
df.c .= uppercase.(df.c) # Convert all strings in column 'c' to uppercase

Broadcasting assignment works with sub-selections as well:

In [ ]:
df[1:2, :a] .= 99 # Set the first two values of column 'a' to 99

# Story: Working with real data

We'll now look at a more in-depth, hands-on exercise of using DataFrames.

The Queensland government has an open data portal, and makes available [tide predictions at various locations](https://www.data.qld.gov.au/dataset/coastal-data-system-tide-data/resource/1311fc19-1e60-444f-b5cf-24687f1c15a7) on the state's coast. (There's some other interesting data as well at <https://www.qld.gov.au/tides>).

Let's use this to do some exploration. We'll first download with the HTTP.jl package and write it to `tides.csv`

In [ ]:
using HTTP

response = HTTP.get("https://www.data.qld.gov.au/datastore/dump/1311fc19-1e60-444f-b5cf-24687f1c15a7?bom=True")
write("work/tides.csv", response.body)

Let's explore the first few lines

In [ ]:
open("work/tides.csv") do io
    for i ∈ 1:5
        line = readline(io)
        println(line)
    end
end

We can read it into a dataframe with `CSV.read`, and show the first few lines with `first`

In [ ]:
using CSV

df = CSV.read("work/tides.csv", DataFrame)
first(df, 5)

Note the inferred datatypes, including the automatically converted `DateTime`. We can customize this

In [ ]:
# we could also do
df32 = CSV.read("work/tides.csv", DataFrame; types=Dict("Water Level" => Float32, "Prediction" => Float32, "Residual" => Float32, "Latitude" => Float32, "Longitude" => Float32));

In [ ]:
println("With Float32s, we saved $(round((1-Base.summarysize(df32)/Base.summarysize(df))*100; digits=2))% memory")

(This is silly, don't do it in practice.)

Let's look also at the last rows

In [ ]:
last(df, 3)

Here it seems that "-99.0" seems to mean missing. Let's see where it's coming from in the CSV

In [ ]:
open("work/tides.csv") do io
    while true
        line = readline(io)
        if contains(line, "-99")
            println(line)
            break
        end
    end
end

We can tell `CSV.read` to mark values with "-99.000" as `missing`

In [ ]:
df = CSV.read("work/tides.csv", DataFrame; missingstring=["-99.000"])
last(df, 3)

Note the "?" in water level/residual: this is DataFrames notation for columns which contain missing data.

Referring to `Water Level` is a bit annoying now:

In [ ]:
df[:, Symbol("Water Level")]

Let's rename it, and let's rename `DateTime` too to avoid confusion:

In [ ]:
# ! means in-place
rename!(df, Symbol("Water Level") => :WaterLevel, Symbol("DateTime") => :Time)
first(df, 5)

Drop some redundant columns

In [ ]:
select!(df, [:Site, :Latitude, :Longitude, :Time, :WaterLevel, :Prediction])
first(df, 5)

Here is our list of columns:

In [ ]:
names(df)

Or by piping

In [ ]:
df |> names

## Getting to know our data

Let's dive a bit deeper, what do we have?

In [ ]:
describe(df)

What are the site names?

In [ ]:
unique(df.Site)

A note on `String15`:

In [ ]:
df.Site

Let's group by site

In [ ]:
# groupby takes a dataframe and a list of columns to group by
by_site = groupby(df, :Site)

This produces a grouped dataframe

In [ ]:
typeof(by_site)

What's the mean water level per site?

In [ ]:
# get the mean function
using Statistics

# enter ∘ with \circ TAB
# combine takes the grouped df and a list of operations
combine(by_site, :WaterLevel => mean ∘ skipmissing => :MeanWaterLevel)

Here we applied `mean(skipmissing(...))` to the `:WaterLevel` column.

Let's plot the water level at some sites

In [ ]:
using Plots

my_sites = ["coombabahst", "russellislande", "rabybay"]

p = plot(
    xlabel="Time",
    ylabel="Water Level",
    title="Water Level Over Time for Selected Sites",
    legend=:topleft
)

for group in by_site
    site_name = group.Site[1] # Get the site name from the first row of the group
    if site_name ∉ my_sites
        continue
    end
    plot!(
        p,
        group.Time,
        group.WaterLevel,
        # Label for the legend
        label=site_name,
        linealpha=0.8,
        linewidth=2
    )
end

p

How many data points do we have per site?

In [ ]:
combine(by_site, nrow => :Count)

Let's compute the squared residual:

In [ ]:
df[!, :SqResidual] = (df.WaterLevel - df.Prediction).^2
# or with transform
# transform(df, [:WaterLevel, :Prediction] => ((w, p) -> (w-p).^2) => :c)

There were some sites with fully missing water levels

In [ ]:
all_missing = combine(groupby(df, :Site), :WaterLevel => (x -> all(ismissing, x)) => :IsMissing)

In [ ]:
filter!(row -> row.IsMissing == false, all_missing)

In [ ]:
select!(all_missing, Not(:IsMissing))

In [ ]:
df_clean = innerjoin(df, all_missing, on=:Site)

Let's compute the 90th percentile of water level per site?

In [ ]:
p90(x) = quantile(x, .9)
combine(groupby(df_clean, :Site), :WaterLevel => p90 ∘ skipmissing => :WaterLevelP90)

Let's plot the mean square error in prediction per site

In [ ]:
mse_by_site = combine(groupby(df_clean, :Site), :SqResidual => mean ∘ skipmissing => :MSE)

plot(mse_by_site.Site, mse_by_site.MSE, seriestype=:bar, xrotation=45, title="MSE in water level prediction by site")

## More Transformations

Here are key operations:

- [`groupby`](https://dataframes.juliadata.org/stable/lib/functions/#Base.groupby) – Split a DataFrame into groups by one or more columns.
- [`combine`](https://dataframes.juliadata.org/stable/man/split_apply_combine/#Combining-results) – Apply functions to groups or columns and combine results in a new DataFrame.
- [`transform`](https://dataframes.juliadata.org/stable/man/getting_started/#Transforming-data) – Create or modify columns (optionally in-place).
- [`select`](https://dataframes.juliadata.org/stable/man/getting_started/#Selecting-columns) – Select (and transform) columns, optionally creating new ones.

**With [DataFramesMeta.jl](https://juliadata.github.io/DataFramesMeta.jl/stable/):**
- [`@subset`](https://juliadata.github.io/DataFramesMeta.jl/stable/macros/#@subset) – Filter rows based on row-wise conditions.
- [`@select`](https://juliadata.github.io/DataFramesMeta.jl/stable/macros/#@select) – Select or transform columns.
- [`@transform`](https://juliadata.github.io/DataFramesMeta.jl/stable/macros/#@transform) – Add or modify columns by assignment.
- [`@combine`](https://juliadata.github.io/DataFramesMeta.jl/stable/macros/#@combine) – Combine results of group operations into a DataFrame.

For more, see the official [DataFrames.jl documentation](https://dataframes.juliadata.org/stable/) and the [DataFramesMeta.jl documentation](https://juliadata.github.io/DataFramesMeta.jl/stable/).

There are some other great resources on the web:

1. [A PumasAI tutorial](https://tutorials.pumas.ai/html/DataWranglingInJulia/05-mutating-dfmeta.html)
1. [A UQ course tutorial](https://courses.smp.uq.edu.au/MATH2504/2025/lectures_html/lecture-unit-8.html)

## More dataframes

Here are the common packages in this ecosystem:

* [DataFrames.jl](https://github.com/JuliaData/DataFrames.jl) - the main dataframes package.
* [DataFramesMeta.jl](https://github.com/JuliaData/DataFramesMeta.jl) - metaprogramming tools for DataFrames.jl objects.
* [CSV.jl](https://github.com/JuliaData/CSV.jl) - read and write to CSV files.
* [CategoricalArrays.jl](https://github.com/JuliaData/CategoricalArrays.jl) - provides tools for working with categorical variables, both with unordered (nominal variables) and ordered categories (ordinal variables), optionally with missing values.
* [Chain.jl](https://github.com/jkrumbiegel/Chain.jl) - provides a useful macro rewrites a series of expressions into a chain.
* [XLSX.jl](https://github.com/felipenoris/XLSX.jl) - Excel file reader/writer for the Julia language.
* [SummaryTables.jl](https://github.com/PumasAI/SummaryTables.jl) - creating publication-ready tables in HTML, docx, LaTeX and Typst formats.


# JSON

There are two competing [JSON](https://en.wikipedia.org/wiki/JSON) libraries: [JSON.jl](https://github.com/JuliaIO/JSON.jl) and [JSON3.jl](https://github.com/quinnj/JSON3.jl). Here is a `JSON.jl` example:

In [ ]:
using HTTP
using JSON

response = HTTP.get("https://couchers.org/api/status")
data = JSON.parse(String(response.body))

println(data)

# Serialization

Julia provides out of the box [serialization](https://docs.julialang.org/en/v1/stdlib/Serialization/). Here is an example. The example is slightly interesting because we also create a tree data structure. 

<!-- * struct (mutable & immutable)
* write to file
* read from file -->


In [ ]:
using Random

Random.seed!(0)

struct Node
    id::UInt16
    friends::Vector{Node}

    # inner constructor, uses the default constructor
    Node() = new(rand(UInt16), [])
    # another inner constructor
    Node(friend::Node) = new(rand(UInt16),[friend])
end

In [ ]:
"""
Makes `n` children to node, each with a single friend
"""
function make_children(node::Node, n::Int, friend::Node)
    for _ in 1:n
        new_node = Node(friend)
        push!(node.friends, new_node)
    end
end;

In [ ]:
# make a tree
root = Node()
make_children(root, 3, root)
for node in root.friends
    make_children(node, 2,root)
end

root

Note that when we try to show `root`, it's complete gibberish. We can write a `Base.show()` function to make this pretty:

In [ ]:
# make it show up pretty
function Base.show(io::IO, x::Node)
    shown = Set{Node}()
    function recursive_show(y::Node, depth::Int)
        print(io, "  "^depth*"Node: $(y.id)")
        if y in shown
            println(io, " (already shown)")
        else
            push!(shown, y)
            println(io, ", friends:")
            for f in y.friends
                recursive_show(f, depth+1)
            end
        end
    end
    recursive_show(x, 0)
    return nothing
end

root

Suppose we now want to save this in a file...

In [ ]:
using Serialization
serialize("work/tree.dat", root)

In [ ]:
newroot = deserialize("work/tree.dat")

# Additional online resources

* Basic blog post about [Tuples and Vectors, Allocations and Performance for Beginners
](https://jkrumbiegel.com/pages/2020-10-31-tuples-and-vectors/).
* This [introductory post](https://blog.glcs.io/staticarrays) covers [StaticArrays.jl](https://github.com/JuliaArrays/StaticArrays.jl) which is a popular library used in many numerical packages.
* The book @storopolihuijzeralonso2021juliadatascience is a good resource for DataFrames.  
* A paper about `DataFrames.jl` made it into the Journal of Statistical Software, @bouchet2023dataframes.
* A nice "cheatsheet" for dataframes [here](https://www.ahsmart.com/assets/pages/data-wrangling-with-data-frames-jl-cheat-sheet/DataFramesCheatSheet_v1.x_rev1.pdf).
* The book @kaminski2023julia is also written by the main creator of `DataFrames.jl`.
* This [Pumas `DataFramesMeta.jl` tutorial](https://tutorials.pumas.ai/html/DataWranglingInJulia/05-mutating-dfmeta.html) is useful.
* See this [Hacker News](https://news.ycombinator.com/item?id=29360221) discussion around [JSON.jl](https://github.com/JuliaIO/JSON.jl) and [JSON3.jl](https://github.com/quinnj/JSON3.jl).
* This unit used many macros. The proper documentation is [here](https://docs.julialang.org/en/v1/manual/metaprogramming/). This [blog post](https://jkrumbiegel.com/pages/2021-06-07-macros-for-beginners/) is an elementary introduction.

# Exercises

1. You have this dictionary:
```
        country_capital = Dict(
                                "France" => "Paris",
                                "Germany" => "Berlin",
                                "Italy" => "Rome",
                                "Spain" => "Madrid")
```
Now create a new dictionary, `capital_country` where the keys are the capital cities and the values are the country names.

2. Looking up with the `in` or `∈` symbol is possible both in an array and a set. You can create an array with `rand(1:10^10, 10^7)` which will have $10^7$ entries, selected from the numbers $1,\ldots,10^{10}$. You can also wrap this to create a set. Now compare lookup timings with `@time` or `@btime` (from BenchmarkTools.jl) for lookup to see if a single `rand(1:10^10)` is an element of the set.
2. Given the string `text = "Julia is a high-level, high-performance programming language."`, write Julia code to count how many times the substring "high" appears in the text (case-insensitive).
2. Install the `Rdatasets.jl` package. Then load the "iris" dataset. Then, filter the DataFrame to only include rows where the SepalLength is greater than its mean, and display the first five rows of the result.
2. Load the "mtcars" dataset from `RDatasets`. Then, group the data by the `Cyl` (number of cylinders) column and compute the average `MPG` (miles per gallon) for each group. Display the resulting summary DataFrame.
2. Consider this JSON file (put it in a string):
```
                {
                  "name": "Alice",
                  "age": 30,
                  "skills": ["Julia", "Python", "SQL"]
                }
```
Given the JSON string above, write Julia code to parse it and print the person's name and the number of skills they have.

7. Create an array of $10^6$ random `Float64` (you can use `rand(Float64, 3)`). Then serialize and inspect the file size. See it makes sense with `sizeof(Float64)`. Now do the same with `Float16`, `Float32`, `UInt8`, and another type of your choice.
